<a target="_blank" href="https://colab.research.google.com/github/yasstake/rusty-bot/blob/main/manual/market.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Market クラス解説

Marketクラスは、取引所で扱われる通貨ペアーの１組合わせをモデリングしたクラスです。
現在はBinanceの USDBUSDのSPOT取引しか扱えませんが、比較的容易に拡張できるようにつくっていますので順次拡張していきます。

## Marketクラスメソッド

### マーケットオブジェクトの生成(`open`)

`open`メソッドでMaketオブジェクトを開きます。内部的にキャッシュされているので２回目からはキャッシュを返します。そのため複数回呼び出してOKです。

今後SPOT以外にも対応した場合、パラメータが変更になる可能性があります。
```
binance = Market.open(
    'BN',               # Binanceは 'BN'
    'BTCBUSD'
)
```

### データのダウンロード(`download`)

指定した日付あらあとのデータをダウンロードしてDBへ保存します。
BTCBUSDしか対応していないため効果はありませんが、`open`で開いたマーケットオブジェクト全てに対してダウンロード処理をします。

```
Market.download(
    5   #ダウンロード日数（N日前を指定)
    )
```


## Marketオブジェクトメソッド

Marketインスタンスが`binance`オブジェクトに保存されている前提で説明します。

###　情報表示　

####　`_repr_html_()`

jupyuter note上でオブジェクトを入力してリターンすると内部的に`_repr_html_()`がよばれて内容が分かりやすく表示されます。
```
binance
```

#### DBのパス表示 `file_path`

DBは巨大になりますので、不要になったら`file_path`プロパティで表示される場所を確認して手動で消してください。

```
binance.file_name
```

### OHLCV取得（pd.DataFrame形式)

時刻はμsで指定する。開始時刻に0を指定するとDBの最初から、終了に0を指定するとDBの最後まで（現在時刻に一番ちかいときまで）を指定したことになる。

```
binance.ohlcv(
    start_timestamp_us,     #開始時刻（μs）
    end_timestamp_us,       #終了時刻（μs)
    60                      #足の幅（秒）
    )
```

pd.DataFrame 形式でOHLCVデータが取得できます。この機能だけつかっても便利に使えます。
なお、Tickデータから任意の足を再生成しているので初回キャッシュされていないときはかなり時間がかかります。




## 以下動かしながら説明

### Google Collab向けパッケージインストール

In [2]:
# Google Clab用です。必要に応じてコメントアウトしてください
! pip install --upgrade pip
! pip install numpy
! pip install pandas
! pip uninstall -y panel
! pip install --upgrade bokeh >= 3

# https://github.com/yasstake/rusty-bot/releases/
# （環境にあわせて修正お願いします。404エラーが出た場合は新しいリリースがでいるので確認してください）
# Linux/Google Collab用
!  pip install https://github.com/yasstake/rusty-bot/releases/download/release-0.2.0a/rbot-0.2.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 105.2 MB/s eta 0:00:00


### 以後Marktライブラリの動作

In [3]:
# 必要ライブラリのimport
import rbot
from rbot import Market
from rbot import DAYS_BEFORE

In [4]:
# Binance('BN')の'BTCBUSD'ペアーの取引マーケットを生成
binance = Market.open('BN', 'BTCBUSD')

# 上記でオープンしたマーケットに対し5日より新しいログをダウンロード
Market.download(5)

# Market.download(5, True)       #ディフォルトではキャッシュされているので、再ダウンロードしたい場合は強制フラグをTrueにする。Collabでは１日あたり1分ぐらいかかります。

In [5]:
# Marketオブジェクトが管理するDBに保存されているデータ期間を表示する。
binance

start,end
1671148800221000,1671667199909000
"""2022-12-16T00:00:00.221000""","""2022-12-21T23:59:59.909000"""
days=,5


In [6]:
# データベースのDisk上の場所を表示する。不要になったら手で消してください。

binance.file_name

'/root/.local/share/rbot/DB/BN/SPOT-BTCBUSD.db'

## OHLCVの取得

In [7]:
# 初回は遅いです。２回目はキャッシュされているので比較的高速です
binance.ohlcv(
    DAYS_BEFORE(5),     # 5日前から
    0,                  # 最新まで
    60*60
)

,open,high,low,close,volume
timestamp,,,,,
2022-12-17 06:00:00+00:00,16729.13,16776.64,16706.27,16714.89,2770.83525
2022-12-17 07:00:00+00:00,16714.89,16753.80,16679.97,16750.25,3199.92340
2022-12-17 08:00:00+00:00,16750.39,16758.94,16712.00,16723.05,2753.37501
2022-12-17 09:00:00+00:00,16722.95,16739.38,16678.36,16699.33,3410.79708
2022-12-17 10:00:00+00:00,16700.37,16762.10,16666.66,16735.20,3513.70912
...,...,...,...,...,...
2022-12-21 19:00:00+00:00,16789.65,16806.64,16729.61,16756.34,3662.99697
2022-12-21 20:00:00+00:00,16756.48,16783.41,16731.28,16781.68,3504.26428
2022-12-21 21:00:00+00:00,16781.69,16801.83,16754.33,16793.20,2657.77402


短めの足では任意のサイズの足が作れるので、このOHCLVだけを利用して独自の分析を行うにも便利だと思います。
週足より長い足も作れますが、性能が出ないのとデータが揃わない点に問題があり実用できではないかもしれません。

ご活用ください。